<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Ingeniería de Características</h1>

<h3>Proyecto integrador (parte 2): Limpieza de los datos </h3>

<p> Alumno </p>
<p> Fernando Luna Ponce </p>

<p> Profesor </p>
<p> Dr. Julio Waissman Vilanova </p>

</center>

En esta parte del proyecto estaremos aplicando:
- Armonización de variables
- Manejo y codificación de valores cuantitativos, fechas
- Manejo de valores perdidos
- Detección de y manejo de valores anómalos

Importamos las librerías necesarias.

In [123]:
import pandas as pd # Solo para ver el archivo descargado
import numpy as np
import datetime

# Limpieza de datos

Nuestra raw data la importamos para comenzar con la limpieza.

In [124]:
subdir = "./data/raw_data/"

rawDataDefunciones2017 = pd.read_parquet(subdir + 'raw_data_Defunciones2017.parquet')
rawDataDefunciones2018 = pd.read_parquet(subdir + 'raw_data_Defunciones2018.parquet')
rawDataDefunciones2019 = pd.read_parquet(subdir + 'raw_data_Defunciones2019.parquet')
rawDataDefunciones2020 = pd.read_parquet(subdir + 'raw_data_Defunciones2020.parquet')
rawDataDefunciones2021 = pd.read_parquet(subdir + 'raw_data_Defunciones2021.parquet')
rawDataHabitantes = pd.read_parquet(subdir + 'raw_data_Habitantes.parquet')

1. Filtrar por Estado de residencia habitual del fallecido, ya que este campo es el que nos indica donde radicaba, en este caso solo nos interesan las correspondientes al Estado de Sonora (26)

In [125]:
rawDataDefunciones2017 = rawDataDefunciones2017[rawDataDefunciones2017["ent_resid"] == 26]
print(rawDataDefunciones2017.shape)

rawDataDefunciones2018 = rawDataDefunciones2018[rawDataDefunciones2018["ent_resid"] == 26]
print(rawDataDefunciones2018.shape)

rawDataDefunciones2019 = rawDataDefunciones2019[rawDataDefunciones2019["ent_resid"] == 26]
print(rawDataDefunciones2019.shape)

rawDataDefunciones2020 = rawDataDefunciones2020[rawDataDefunciones2020["ent_resid"] == 26]
print(rawDataDefunciones2020.shape)

rawDataDefunciones2021 = rawDataDefunciones2021[rawDataDefunciones2021["ent_resid"] == 26]
print(rawDataDefunciones2021.shape)

(16729, 59)
(17644, 62)
(18485, 59)
(27814, 59)
(25273, 59)


2. Filtrar solo los que tengan causa de defunción por diabetes, para esto tenemos abrimos otro excel con la lista de nuestros valores para la columna de causa de defunción. Vamos a extraer estos valores y con estos filtraremos nuestro dataframe.

In [126]:
causaDef2017 = pd.read_csv('./data/fuente_1/catalogos/decatcausa.csv', encoding='latin-1')
causaDiabetes2017 = causaDef2017.query('DESCRIP.str.contains("Diabetes")', engine='python')

causaDef2018 = pd.read_csv('./data/fuente_2/catalogos/causa_defuncion.csv', encoding='latin-1')
causaDiabetes2018 = causaDef2018.query('DESCRIP.str.contains("Diabetes")', engine='python')

causaDef2019 = pd.read_csv('./data/fuente_3/catalogos/causa_defuncion.csv', encoding='latin-1')
causaDiabetes2019 = causaDef2019.query('descrip.str.contains("Diabetes")', engine='python')

causaDef2020 = pd.read_csv('./data/fuente_4/catalogos/causa_defuncion.csv', encoding='latin-1')
causaDiabetes2020 = causaDef2020.query('descrip.str.contains("Diabetes")', engine='python')

causaDef2021 = pd.read_csv('./data/fuente_5/catalogos/causa_defuncion.csv', encoding='latin-1')
causaDiabetes2021 = causaDef2021.query('descrip.str.contains("Diabetes")', engine='python')

In [127]:
rawDataDefunciones2017 = rawDataDefunciones2017[rawDataDefunciones2017['causa_def'].isin(causaDiabetes2017["CVE"])]
print(rawDataDefunciones2017.shape)

rawDataDefunciones2018 = rawDataDefunciones2018[rawDataDefunciones2018['causa_def'].isin(causaDiabetes2018["CVE"])]
print(rawDataDefunciones2018.shape)

rawDataDefunciones2019 = rawDataDefunciones2019[rawDataDefunciones2019['causa_def'].isin(causaDiabetes2019["cve"])]
print(rawDataDefunciones2019.shape)

rawDataDefunciones2020 = rawDataDefunciones2020[rawDataDefunciones2020['causa_def'].isin(causaDiabetes2020["cve"])]
print(rawDataDefunciones2020.shape)

rawDataDefunciones2021 = rawDataDefunciones2021[rawDataDefunciones2021['causa_def'].isin(causaDiabetes2021["cve"])]
print(rawDataDefunciones2021.shape)

(1828, 59)
(1545, 62)
(1614, 59)
(2268, 59)
(1784, 59)


3. Unimos los dataframes, pero para esto primero tenemos que dejar estos con la misma cantidad de columnas:

In [128]:
columnas = rawDataDefunciones2021.columns
rawDataDefunciones2017 = rawDataDefunciones2017[columnas]
print(rawDataDefunciones2017.shape)

rawDataDefunciones2018 = rawDataDefunciones2018[columnas]
print(rawDataDefunciones2018.shape)

rawDataDefunciones2019 = rawDataDefunciones2019[columnas]
print(rawDataDefunciones2019.shape)

rawDataDefunciones2020 = rawDataDefunciones2020[columnas]
print(rawDataDefunciones2020.shape)

(1828, 59)
(1545, 59)
(1614, 59)
(2268, 59)


In [129]:
rawDataDefunciones = pd.concat([rawDataDefunciones2017, rawDataDefunciones2018, rawDataDefunciones2019, rawDataDefunciones2020, rawDataDefunciones2021])
rawDataDefunciones.shape

(9039, 59)

Generamos una columna de datetime

In [130]:
#rawDataDefunciones = rawDataDefunciones[rawDataDefunciones['dia_regis']<=31]
rawDataDefunciones['dia_regis'] = rawDataDefunciones['dia_regis'].apply(lambda x: 28 if x > 31 else x)
print(rawDataDefunciones.shape)

(9039, 59)


In [131]:
rawDataDefunciones['fechaRegistro'] = rawDataDefunciones['anio_regis'].astype(str) + rawDataDefunciones['mes_regis'].apply(lambda x: '{0:0>2}'.format(x)) + rawDataDefunciones['dia_regis'].apply(lambda x: '{0:0>2}'.format(x))
rawDataDefunciones['fechaRegistro'] = pd.to_datetime(rawDataDefunciones['fechaRegistro'])
rawDataDefunciones.head()


,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,...,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax,fechaRegistro
6468,2,2,26,55,13,1,2,2,15,1,...,None,9,9,88,88,888,8888,0,999,2017-02-24
6489,2,2,26,55,13,1,2,2,15,1,...,None,9,1,88,88,888,8888,0,999,2017-02-20
6612,2,2,26,55,13,1,2,2,15,1,...,None,9,1,88,88,888,8888,0,999,2017-01-20
8834,2,2,26,55,13,1,2,2,15,1,...,None,9,2,88,88,888,8888,0,999,2017-01-09
9278,2,2,26,55,13,1,2,2,15,1,...,None,1,2,88,88,888,8888,0,999,2017-02-11


In [132]:
rawDataDefunciones = rawDataDefunciones[["mun_resid","fechaRegistro", "tloc_resid", "ocupacion", "sexo", "edad", "asist_medi", "sitio_ocur", "lengua"]]
rawDataDefunciones.head()

,mun_resid,fechaRegistro,tloc_resid,ocupacion,sexo,edad,asist_medi,sitio_ocur,lengua
6468,55,2017-02-24,13,99,1,4072,1,4,9
6489,55,2017-02-20,13,8,1,4028,1,3,9
6612,55,2017-01-20,13,9,1,4056,1,3,9
8834,55,2017-01-09,13,11,2,4078,1,9,9
9278,55,2017-02-11,13,11,2,4071,1,3,1


Revisamos los tipos de datos

In [133]:
rawDataDefunciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9039 entries, 6468 to 950254
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   mun_resid      9039 non-null   int64         
 1   fechaRegistro  9039 non-null   datetime64[ns]
 2   tloc_resid     9039 non-null   int64         
 3   ocupacion      9039 non-null   int64         
 4   sexo           9039 non-null   int64         
 5   edad           9039 non-null   int64         
 6   asist_medi     9039 non-null   int64         
 7   sitio_ocur     9039 non-null   int64         
 8   lengua         9039 non-null   int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 706.2 KB


Vamos a generar columnas con las descripciones que corresponden a algunas variables, ya que estas están representadas con números pero realmente son variables categóricas

In [134]:
rawDataHabitantes["Codigo_Municipio"] = pd.to_numeric(rawDataHabitantes["Codigo_Municipio"])
df_1 = pd.merge(rawDataDefunciones, rawDataHabitantes, how='left', left_on='mun_resid', right_on='Codigo_Municipio')

In [135]:
ruta_archivos = "./data/fuente_5/catalogos/"
tamanio_loc = pd.read_csv(ruta_archivos + 'tamaño_localidad.csv', encoding='latin-1')
df_2 = pd.merge(df_1, tamanio_loc, how='left', left_on='tloc_resid', right_on='CVE')
df_2.rename(columns={ df_2.columns[13]: "tam_localidad" }, inplace = True)


In [136]:
ruta_archivos = "./data/fuente_5/catalogos/"
ocupacion = pd.read_csv(ruta_archivos + 'ocupacion.csv', encoding='latin-1')
df_3 = pd.merge(df_2, ocupacion, how='left', left_on='ocupacion', right_on='CVE')
df_3.rename(columns={ df_3.columns[15]: "descr_ocupacion" }, inplace = True)


In [137]:
ruta_archivos = "./data/fuente_5/catalogos/"
sexo = pd.read_csv(ruta_archivos + 'sexo.csv', encoding='latin-1')
df_4 = pd.merge(df_3, sexo, how='left', left_on='sexo', right_on='CVE')
df_4.rename(columns={ df_4.columns[17]: "descr_sexo" }, inplace = True)

In [138]:
ruta_archivos = "./data/fuente_5/catalogos/"
asist_medi = pd.read_csv(ruta_archivos + 'asistencia_medica.csv', encoding='latin-1')
df_5 = pd.merge(df_4, asist_medi, how='left', left_on='asist_medi', right_on='CVE')
df_5.rename(columns={ df_5.columns[19]: "descr_asist_medi" }, inplace = True)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3800\3317154665.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'CVE_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_5 = pd.merge(df_4, asist_medi, how='left', left_on='asist_medi', right_on='CVE')


In [139]:
ruta_archivos = "./data/fuente_5/catalogos/"
sitio_ocur = pd.read_csv(ruta_archivos + 'sitio_ocurrencia.csv', encoding='latin-1')
df_6 = pd.merge(df_5, sitio_ocur, how='left', left_on='sitio_ocur', right_on='CVE')
df_6.rename(columns={ df_6.columns[21]: "descr_sitio_ocur" }, inplace = True)

In [140]:
ruta_archivos = "./data/fuente_5/catalogos/"
lengua = pd.read_csv(ruta_archivos + 'lengua_indigena.csv', encoding='latin-1')
df_7 = pd.merge(df_6, lengua, how='left', left_on='lengua', right_on='CVE')
df_7.rename(columns={ df_7.columns[23]: "descr_lengua" }, inplace = True)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3800\837637220.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'CVE_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_7 = pd.merge(df_6, lengua, how='left', left_on='lengua', right_on='CVE')


In [141]:
df_8 = df_7[df_7['mun_resid']!=999]
df_8.rename(columns={ df_8.columns[12]: "x" }, inplace = True)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3800\2058429356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_8.rename(columns={ df_8.columns[12]: "x" }, inplace = True)


In [142]:
valores = [4998] # Valor no especificado
df_8 = df_8[-df_8["edad"].isin(valores)] # Agregamos el simbolo "-" para filtrar los valores que no estén en el df
df_8["Edad"] = df_8['edad'].apply(lambda x: 1 if x <= 4001 else x - 4000)
df_8

,mun_resid,fechaRegistro,tloc_resid,ocupacion,sexo,edad,asist_medi,sitio_ocur,lengua,Codigo_Municipio,...,descr_ocupacion,x,descr_sexo,CVE_y,descr_asist_medi,x,descr_sitio_ocur,CVE_y,descr_lengua,Edad
0,55,2017-02-24,13,99,1,4072,1,4,9,55.0,...,No especificada,1,Hombre,1,Con asistencia médica,4,ISSSTE,9,Se ignora,72
1,55,2017-02-20,13,8,1,4028,1,3,9,55.0,...,"Operadores de maquinaria industrial, ensamblad...",1,Hombre,1,Con asistencia médica,3,IMSS,9,Se ignora,28
2,55,2017-01-20,13,9,1,4056,1,3,9,55.0,...,Trabajadores en actividades elementales y de a...,1,Hombre,1,Con asistencia médica,3,IMSS,9,Se ignora,56
3,55,2017-01-09,13,11,2,4078,1,9,9,55.0,...,No trabaja,2,Mujer,1,Con asistencia médica,9,Unidad médica privada,9,Se ignora,78
4,55,2017-02-11,13,11,2,4071,1,3,1,55.0,...,No trabaja,2,Mujer,1,Con asistencia médica,3,IMSS,1,Sí,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9034,18,2021-12-15,14,11,1,4078,1,11,2,18.0,...,No trabaja,1,Hombre,1,Con asistencia médica,11,Hogar,2,No,78
9035,55,2021-12-28,13,5,1,4070,1,11,2,55.0,...,Trabajadores en servicios personales y vigilancia,1,Hombre,1,Con asistencia médica,11,Hogar,2,No,70
9036,42,2021-12-23,2,11,2,4061,2,3,2,42.0,...,No trabaja,2,Mujer,2,Sin asistencia médica,3,IMSS,2,No,61
9037,55,2021-12-15,1,11,2,4061,1,11,2,55.0,...,No trabaja,2,Mujer,1,Con asistencia médica,11,Hogar,2,No,61


In [143]:
rawDataDefunciones = df_8.iloc[:, [1, 10, 2, 3, 17, 24, 6, 7, 8]]
rawDataDefunciones.head()

,fechaRegistro,Nombre_Municipio,tloc_resid,ocupacion,descr_sexo,Edad,asist_medi,sitio_ocur,lengua
0,2017-02-24,San Luis Río Colorado,13,99,Hombre,72,1,4,9
1,2017-02-20,San Luis Río Colorado,13,8,Hombre,28,1,3,9
2,2017-01-20,San Luis Río Colorado,13,9,Hombre,56,1,3,9
3,2017-01-09,San Luis Río Colorado,13,11,Mujer,78,1,9,9
4,2017-02-11,San Luis Río Colorado,13,11,Mujer,71,1,3,1


In [144]:
rawDataDefunciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9034 entries, 0 to 9038
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fechaRegistro     9034 non-null   datetime64[ns]
 1   Nombre_Municipio  9034 non-null   object        
 2   tloc_resid        9034 non-null   int64         
 3   ocupacion         9034 non-null   int64         
 4   descr_sexo        9034 non-null   object        
 5   Edad              9034 non-null   int64         
 6   asist_medi        9034 non-null   int64         
 7   sitio_ocur        9034 non-null   int64         
 8   lengua            9034 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 705.8+ KB


Guardamos nuestro dataframe con datos tidy

In [145]:
subdir = "./data/tidy_data/"
if not os.path.exists(subdir):
        os.makedirs(subdir)
rawDataDefunciones.to_parquet(subdir + 'Tidy_data.parquet')